In [125]:
import pyfredapi as pf
import pandas as pd
import numpy as np
import time
import pickle

In [31]:
API_KEY = '10e0969f13a4b82bc47d736e1047d303'

In [67]:
# Init w/ base categories
category_tree = [
    {'name': 'Money, Banking, & Finance', 'id': 1, 'children': []},
    {'name': 'Population, Employment, & Labor Markets', 'id': 10, 'children': []},
    {'name': 'National Accounts', 'id': 32992, 'children': []},
    {'name': 'Production & Business Activity', 'id': 1, 'children': []},
    {'name': 'Prices', 'id': 32455, 'children': []},
    {'name': 'International Data', 'id': 32263, 'children': []},
    {'name': 'U.S. Regional Data', 'id': 3008, 'children': []},
    {'name': 'Academic Data', 'id': 33060, 'children': []},    
]

In [65]:

def build_category_tree(category_tree):
    
    queue = category_tree.copy()
    while len(queue) > 0:
        elem = queue.pop(0)
        time.sleep(1)
#         print(f'retrieving children for: {elem["name"]}')
        categories = pf.get_category_children(category_id=elem['id'], api_key=API_KEY)
        children = categories['categories']
        elem['children'] = children        
        if len(children) > 0:
            queue += children
    
    return category_tree
            

In [ ]:
filled_category_tree = build_category_tree(category_tree)

In [71]:
# pickle.dump(filled_category_tree, open('/Users/jonathanl/fred-categories.pkl', 'wb'))

In [66]:
def extract_terminal_nodes(category_tree):
    
    terminal_nodes = []
    queue = category_tree.copy()
    while len(queue) > 0:
        elem = queue.pop(0)
#         print(f'visiting node: {elem["name"]}')
        if len(elem['children']) < 1:
            terminal_nodes.append(elem)
        else:
            queue.extend(elem['children'])
    return terminal_nodes

In [ ]:
terminal_nodes = extract_terminal_nodes(filled_category_tree)

In [64]:
len(terminal_nodes)

4655

In [81]:
def build_series_dictionary(terminal_categories):
    '''
    Given list of categories, get associated series
    link each series to its terminal 
    Series will consist of series metadata + actual series data (with all / full revisions)
    '''
    series_frames = []
    for category in terminal_categories:
        print(f'processing category: {category["name"]}')
        category_series = pf.get_category_series(category_id=category['id'], api_key=API_KEY)
        series_df = pd.DataFrame.from_records([vars(series_info) for series_info in category_series.values()])
        series_df['category_id'] = category['id']
        series_frames.append(series_df)
        time.sleep(1)
    
    return series_frames

In [82]:
series_frames = build_series_dictionary(terminal_nodes)

processing category: Business Cycle Expansions & Contractions
processing category: Business Surveys
processing category: Construction
processing category: Emissions
processing category: Finance Companies
processing category: Health Insurance
processing category: Patents
processing category: Retail Trade
processing category: Services
processing category: Technology
processing category: Wholesale Trade
processing category: ADP Employment
processing category: Education
processing category: Income Distribution
processing category: Labor Market Conditions
processing category: Minimum Wage
processing category: Weekly Initial Claims
processing category: Tax Data
processing category: Federal Government Debt
processing category: Business Cycle Expansions & Contractions
processing category: Business Surveys
processing category: Construction
processing category: Emissions
processing category: Finance Companies
processing category: Health Insurance
processing category: Patents
processing category:

processing category: Albania
processing category: Algeria
processing category: American Samoa
processing category: Andorra
processing category: Angola
processing category: Anguilla
processing category: Antigua and Barbuda
processing category: Argentina
processing category: Armenia
processing category: Aruba
processing category: Australia
processing category: Austria
processing category: Azerbaijan
processing category: Bahamas
processing category: Bahrain
processing category: Bangladesh
processing category: Barbados
processing category: Belarus
processing category: Belgium
processing category: Belize
processing category: Benin
processing category: Bermuda
processing category: Bhutan
processing category: Bolivia, Plurinational State of
processing category: Caribbean Netherlands
processing category: Bosnia and Herzegovina
processing category: Botswana
processing category: Brazil
processing category: British Indian Ocean Territory
processing category: Brunei Darussalam
processing category:

processing category: Bureau of Labor Statistics Asian Newly Industrialized Countries
processing category: Bureau of Labor Statistics Industrialized Countries
processing category: Bureau of Labor Statistics Latin America
processing category: Bureau of Labor Statistics Pacific Rim
processing category: Emerging Markets
processing category: Euro Area
processing category: European Economic Area
processing category: European Union
processing category: Four Big European Countries
processing category: Group of Seven (G7)
processing category: Group of Twenty Finance Ministers and Central Bank Governors (G20)
processing category: International Monetary Fund
processing category: International Organizations
processing category: Major 5 Asia
processing category: NAFTA
processing category: Non-OECD
processing category: OECD Non-member Economies
processing category: OECD Total
processing category: OECD + Non-member Economies
processing category: OECD - Europe
processing category: Unidentified Countri

processing category: Nonmetallic Mineral Products
processing category: Processed Foods and Feeds
processing category: Professional Services
processing category: Publishing Sales, Excluding Software
processing category: Pulp, Paper, and Allied Products
processing category: Real Estate Services
processing category: Rental and Leasing of Goods
processing category: Repair and Maintenance Services
processing category: Retail Trade Services
processing category: Rubber and Plastic Products
processing category: Selected Security Services
processing category: Software Publishing
processing category: Special Indexes
processing category: Stage of Processing
processing category: Telecommunication, Cable, and Internet User Services
processing category: Textile Products and Apparel
processing category: Transportation Equipment
processing category: Transportation Services
processing category: Travel Arrangement Services
processing category: Warehousing, Storage, and Related Services
processing catego

processing category: Hotels and Restaurants
processing category: Miscellaneous Goods and Services
processing category: Recreation and Culture
processing category: Special Indexes
processing category: Transport
processing category: Autauga County, AL
processing category: Baldwin County, AL
processing category: Barbour County, AL
processing category: Bibb County, AL
processing category: Blount County, AL
processing category: Bullock County, AL
processing category: Butler County, AL
processing category: Calhoun County, AL
processing category: Chambers County, AL
processing category: Cherokee County, AL
processing category: Chilton County, AL
processing category: Choctaw County, AL
processing category: Clarke County, AL
processing category: Clay County, AL
processing category: Cleburne County, AL
processing category: Coffee County, AL
processing category: Colbert County, AL
processing category: Conecuh County, AL
processing category: Coosa County, AL
processing category: Covington County, 

...

processing category: Roane County, TN
processing category: Robertson County, TN
processing category: Rutherford County, TN
processing category: Scott County, TN
processing category: Sequatchie County, TN
processing category: Sevier County, TN
processing category: Shelby County, TN
processing category: Smith County, TN
processing category: Stewart County, TN
processing category: Sullivan County, TN
processing category: Sumner County, TN
processing category: Tipton County, TN
processing category: Trousdale County, TN
processing category: Unicoi County, TN
processing category: Union County, TN
processing category: Van Buren County, TN
processing category: Warren County, TN
processing category: Washington County, TN
processing category: Wayne County, TN
processing category: Weakley County, TN
processing category: White County, TN
processing category: Williamson County, TN
processing category: Wilson County, TN
processing category: Chattanooga, TN-GA
processing category: Clarksville, TN-KY


processing category: Navarro County, TX
processing category: Newton County, TX
processing category: Nolan County, TX
processing category: Nueces County, TX
processing category: Ochiltree County, TX
processing category: Oldham County, TX
processing category: Orange County, TX
processing category: Palo Pinto County, TX
processing category: Panola County, TX
processing category: Parker County, TX
processing category: Parmer County, TX
processing category: Pecos County, TX
processing category: Polk County, TX
processing category: Potter County, TX
processing category: Presidio County, TX
processing category: Rains County, TX
processing category: Randall County, TX
processing category: Reagan County, TX
processing category: Real County, TX
processing category: Red River County, TX
processing category: Reeves County, TX
processing category: Refugio County, TX
processing category: Roberts County, TX
processing category: Robertson County, TX
processing category: Rockwall County, TX
processing 

processing category: Fairfax City, VA
processing category: Falls Church City, VA
processing category: Fauquier County, VA
processing category: Floyd County, VA
processing category: Fluvanna County, VA
processing category: Franklin County, VA
processing category: Franklin City, VA
processing category: Frederick County, VA
processing category: Fredericksburg City, VA
processing category: Galax City, VA
processing category: Giles County, VA
processing category: Gloucester County, VA
processing category: Goochland County, VA
processing category: Grayson County, VA
processing category: Greene County, VA
processing category: Greensville County, VA
processing category: Halifax County, VA
processing category: Hampton City, VA
processing category: Hanover County, VA
processing category: Harrisonburg City, VA
processing category: Henrico County, VA
processing category: Henry County, VA
processing category: Highland County, VA
processing category: Hopewell City, VA
processing category: Isle of Wi

processing category: Beckley, WV
processing category: Charleston, WV
processing category: Cumberland, MD-WV
processing category: Hagerstown-Martinsburg, MD-WV
processing category: Huntington-Ashland, WV-KY-OH
processing category: Morgantown, WV
processing category: Parkersburg-Vienna, WV
processing category: Washington-Arlington-Alexandria, DC-VA-MD-WV
processing category: Weirton-Steubenville, WV-OH
processing category: Wheeling, WV-OH
processing category: Winchester, VA-WV
processing category: Washington-Baltimore, DC-MD-VA-WV (CMSA)
processing category: Washington-Arlington-Alexandria, DC-VA-MD-WV
processing category: Adams County, WA
processing category: Asotin County, WA
processing category: Benton County, WA
processing category: Chelan County, WA
processing category: Clallam County, WA
processing category: Clark County, WA
processing category: Columbia County, WA
processing category: Cowlitz County, WA
processing category: Douglas County, WA
processing category: Ferry County, WA


processing category: EC and National Indicators
processing category: EC Indicator
processing category: EC and National Indicators
processing category: EC Indicator
processing category: EC Indicator
processing category: EC and National Indicators
processing category: EC and National Indicators
processing category: EC and National Indicators
processing category: EC and National Indicators
processing category: National Indicator
processing category: EC and National Indicators
processing category: EC Indicator
processing category: EC Indicator
processing category: EC Indicator
processing category: EC and National Indicators
processing category: National Indicator
processing category: Fremont (Includes Yellowstone Park) County, ID
processing category: Boston-Cambridge-Newton, MA
processing category: Haverhill-Newburyport-Amesbury Town, MA-NH
processing category: Lawrence-Methuen Town-Salem, MA-NH
processing category: Lowell-Billerica-Chelmsford, MA-NH
processing category: Lynn-Saugus-Marble

In [87]:
series_df = pd.concat(series_frames)

In [89]:
series_df.to_parquet('/Users/jonathanl/Workspace/quant-workspace/experimental-fred/data/series-meta/series-meta.pq')

In [92]:
series_df.id.unique()

585946

In [101]:
series_df.columns

Index(['id', 'realtime_start', 'realtime_end', 'title', 'observation_start',
       'observation_end', 'frequency', 'frequency_short', 'units',
       'units_short', 'seasonal_adjustment', 'seasonal_adjustment_short',
       'last_updated', 'popularity', 'notes', 'group_popularity',
       'category_id'],
      dtype='object')

In [117]:
excluded_child_categories = pickle.load(open('/Users/jonathanl/Workspace/quant-workspace/experimental-fred/data/regional-categories.pkl', 'rb'))

In [118]:
len(excluded_child_categories)

4212

In [123]:
series_filtered_df = series_df[~series_df.category_id.isin(excluded_child_categories)]

In [124]:
len(series_filtered_df.id.unique())

156469

In [146]:
def get_series_full(series_id: str) -> pd.DataFrame:
    '''
    full data, all revisions
    add series id to dataframe
    '''    
    try:
        data = pf.get_series_all_releases(series_id, api_key=API_KEY)
        data['series'] = series_id
        return data
    except:
        print(f'!! failed to get: {series_id} !!')
        return None
    
# TODO: retrieve incremental data / information
# Figure out release schedule / release logistics


In [134]:
series_grouped = np.array_split(series_filtered_df.id.unique(), 1000)

In [140]:
# all series dataframes
series_df = []

In [158]:

# process by group, avoid issues
idx = 53
for group in series_grouped[53:]:
    print(f"processing group: {idx}")    
    for series_id in group:        
        print(f"{series_id}, ", end="")
        series_data = get_series_full(series_id)
        if series_data is not None:
            series_df.append(series_data)
        time.sleep(1)
    print()    
    idx += 1
    

processing group: 53
DIBLBOPIBLEND, DIBLBOPIMEPS, DICISYPIBLEND, DICISYPIMEPS, DIDISYPIBLEND, DIDISYPIMEPS, DIGESYPIBLEND, DIGESYPIMEPS, DIMSCTPIBLEND, DIMSCTPIMEPS, DIRESYPIBLEND, DIRESYPIMEPS, DISKSOPIBLEND, DISKSOPIMEPS, DISOTHPIBLEND, DISOTHPIMEPS, DONSSOPIBLEND, DONSSOPIMEPS, EDCNMIPIBLEND, EDCNMIPIMEPS, HLTHSCPIBLEND, HLTHSCPIMEPS, HLTHSEPIBLEND, HLTHSEPIMEPS, INFAPSPIBLEND, INFAPSPIMEPS, INJPOIPIBLEND, INJPOIPIMEPS, MDSBDSPIBLEND, MDSBDSPIMEPS, MEAPEQPIHCSA, MESEPRPIHCSA, MNINEIPIBLEND, MNINEIPIMEPS, NONDRGPIHCSA, NONUHOPIHCSA, NPSMSMPIBLEND, NPSMSMPIMEPS, NURHOMPIHCSA, OTMEPRPIHCSA, PHMEPRPIHCSA, PHPRDRPIHCSA, PRGONUPIHCSA, RECOUNPIBLEND, RECOUNPIMEPS, SYSIILPIBLEND, SYSIILPIMEPS, THAPEQPIHCSA, THMEEQPIHCSA, AKSTHPI, AKUCSFRCONDOSMSAMID, ALSTHPI, ALUCSFRCONDOSMSAMID, ARSTHPI, ARUCSFRCONDOSMSAMID, ATNHPIUS10180Q, ATNHPIUS10420Q, ATNHPIUS10500Q, ATNHPIUS10580Q, ATNHPIUS10740Q, ATNHPIUS10780Q, ATNHPIUS10900Q, ATNHPIUS11020Q, ATNHPIUS11100Q, ATNHPIUS11180Q, ATNHPIUS11260Q, ATNHPIUS

FLSTHPI, FLUCSFRCONDOSMSAMID, GASTHPI, GAUCSFRCONDOSMSAMID, HISTHPI, HIUCSFRCONDOSMSAMID, HPIPONM226N, HPIPONM226S, IASTHPI, IAUCSFRCONDOSMSAMID, IDSTHPI, IDUCSFRCONDOSMSAMID, ILSTHPI, ILUCSFRCONDOSMSAMID, INSTHPI, INUCSFRCONDOSMSAMID, KSSTHPI, KSUCSFRCONDOSMSAMID, KYSTHPI, KYUCSFRCONDOSMSAMID, LASTHPI, LAUCSFRCONDOSMSAMID, LVXRHTNSA, LVXRHTSA, LVXRLTNSA, LVXRLTSA, LVXRMTNSA, LVXRMTSA, LVXRNSA, LVXRPSNSA, LVXRSA, LXXRCNSA, LXXRCPSNSA, LXXRCSA, LXXRHTNSA, LXXRHTSA, LXXRLTNSA, LXXRLTSA, LXXRMTNSA, LXXRMTSA, LXXRNSA, LXXRPSNSA, LXXRSA, MASTHPI, MAUCSFRCONDOSMSAMID, MDSTHPI, MDUCSFRCONDOSMSAMID, MESTHPI, MEUCSFRCONDOSMSAMID, MISTHPI, MIUCSFRCONDOSMSAMID, MIXRHTNSA, MIXRHTSA, MIXRLTNSA, MIXRLTSA, MIXRMTNSA, MIXRMTSA, MIXRNSA, MIXRPSNSA, MIXRSA, MNSTHPI, MNUCSFRCONDOSMSAMID, MNXRHTNSA, MNXRHTSA, MNXRLTNSA, MNXRLTSA, MNXRMTNSA, MNXRMTSA, MNXRNSA, MNXRPSNSA, MNXRSA, MOSTHPI, MOUCSFRCONDOSMSAMID, MSSTHPI, MSUCSFRCONDOSMSAMID, MTSTHPI, MTUCSFRCONDOSMSAMID, NCSTHPI, NCUCSFRCONDOSMSAMID, NDSTHPI, 

CXU190904LB0503M, CXU190904LB0504M, CXU190904LB0505M, CXU190904LB0506M, CXU190904LB0507M, CXU190904LB05A1M, CXU190904LB05A2M, CXU190904LB0602M, CXU190904LB0603M, CXU190904LB0604M, CXU190904LB0605M, CXU190904LB0606M, CXU190904LB0607M, CXU190904LB0608M, CXU190904LB0609M, CXU190904LB0610M, CXU190904LB0702M, CXU190904LB0703M, CXU190904LB0704M, CXU190904LB0705M, CXU190904LB0706M, CXU190904LB0707M, CXU190904LB0902M, CXU190904LB0903M, CXU190904LB0904M, CXU190904LB0905M, CXU190904LB1002M, CXU190904LB1003M, CXU190904LB1004M, CXU190904LB1005M, CXU190904LB1102M, CXU190904LB1103M, CXU190904LB1104M, CXU190904LB1105M, CXU190904LB1202M, CXU190904LB1203M, CXU190904LB1204M, CXU190904LB1205M, CXU190904LB1206M, CXU190904LB1207M, CXU190904LB1208M, CXU190904LB1209M, CXU190904LB1210M, CXU190904LB1302M, CXU190904LB1303M, CXU190904LB1304M, CXU190904LB1305M, CXU190904LB1306M, CXU190904LB1307M, CXU190904LB1308M, CXU190904LB1309M, CXU190904LB1402M, CXU190904LB1403M, CXU190904LB1404M, CXU190904LB1405M, CXU190904L

CXU270102LB1609M, CXU270102LB1702M, CXU270102LB1703M, CXU270102LB1704M, CXU270102LB1705M, CXU270102LB1802M, CXU270102LB1803M, CXU270102LB1804M, CXU270102LB1805M, CXU500110LB0101M, CXU500110LB0102M, CXU500110LB0103M, CXU500110LB0104M, CXU500110LB0105M, CXU500110LB0106M, CXU500110LB01A1M, CXU500110LB01A2M, CXU500110LB0202M, CXU500110LB0203M, CXU500110LB0204M, CXU500110LB0205M, CXU500110LB0206M, CXU500110LB0207M, CXU500110LB0208M, CXU500110LB0209M, CXU500110LB0210M, CXU500110LB0211M, CXU500110LB0212M, CXU500110LB0213M, CXU500110LB0214M, CXU500110LB0215M, CXU500110LB0216M, CXU500110LB0217M, CXU500110LB0218M, CXU500110LB0219M, CXU500110LB0220M, CXU500110LB0221M, CXU500110LB0222M, CXU500110LB0223M, CXU500110LB02A1M, CXU500110LB02A2M, CXU500110LB0402M, CXU500110LB0403M, CXU500110LB0404M, CXU500110LB0405M, CXU500110LB0406M, CXU500110LB0407M, CXU500110LB0408M, CXU500110LB0409M, CXU500110LB0502M, CXU500110LB0503M, CXU500110LB0504M, CXU500110LB0505M, CXU500110LB0506M, CXU500110LB0507M, CXU500110L

CXU900000LB0607M, CXU900000LB0608M, CXU900000LB0609M, CXU900000LB0610M, CXU900000LB0702M, CXU900000LB0703M, CXU900000LB0704M, CXU900000LB0705M, CXU900000LB0706M, CXU900000LB0707M, CXU900000LB0902M, CXU900000LB0903M, CXU900000LB0904M, CXU900000LB0905M, CXU900000LB1002M, CXU900000LB1003M, CXU900000LB1004M, CXU900000LB1005M, CXU900000LB1102M, CXU900000LB1103M, CXU900000LB1104M, CXU900000LB1105M, CXU900000LB1202M, CXU900000LB1203M, CXU900000LB1204M, CXU900000LB1205M, CXU900000LB1206M, CXU900000LB1207M, CXU900000LB1208M, CXU900000LB1209M, CXU900000LB1210M, CXU900000LB1302M, CXU900000LB1303M, CXU900000LB1304M, CXU900000LB1305M, CXU900000LB1306M, CXU900000LB1307M, CXU900000LB1308M, CXU900000LB1309M, CXU900000LB1402M, CXU900000LB1403M, CXU900000LB1404M, CXU900000LB1405M, CXU900000LB1406M, CXU900000LB1407M, CXU900000LB1408M, CXU900000LB1409M, CXU900000LB1502M, CXU900000LB1503M, CXU900000LB1504M, CXU900000LB1505M, CXU900000LB1506M, CXU900000LB1507M, CXU900000LB1508M, CXU900000LB1509M, CXU900000L

...

XTIMVA01FIM664S, XTIMVA01FIM667S, XTIMVA01FIQ188S, XTIMVA01FIQ657S, XTIMVA01FIQ659S, XTIMVA01FIQ664N, XTIMVA01FIQ664S, XTIMVA01FIQ667S, XTIMVA01FRA188S, XTIMVA01FRA657S, XTIMVA01FRA659S, XTIMVA01FRA664N, XTIMVA01FRA664S, XTIMVA01FRA667S, XTIMVA01FRM657S, XTIMVA01FRM659S, XTIMVA01FRM664N, XTIMVA01FRM664S, XTIMVA01FRM667S, XTIMVA01FRQ188S, XTIMVA01FRQ657S, XTIMVA01FRQ659S, XTIMVA01FRQ664N, XTIMVA01FRQ664S, XTIMVA01FRQ667S, XTIMVA01G7A657S, XTIMVA01G7A659S, XTIMVA01G7A667S, XTIMVA01G7M657S, XTIMVA01G7M659S, XTIMVA01G7M667S, XTIMVA01G7Q657S, XTIMVA01G7Q659S, XTIMVA01G7Q667S, XTIMVA01GBA188S, XTIMVA01GBA657S, XTIMVA01GBA659S, XTIMVA01GBA664N, XTIMVA01GBA664S, XTIMVA01GBA667S, XTIMVA01GBM657S, XTIMVA01GBM659S, XTIMVA01GBM664N, XTIMVA01GBM664S, XTIMVA01GBM667S, XTIMVA01GBQ188S, XTIMVA01GBQ657S, XTIMVA01GBQ659S, XTIMVA01GBQ664N, XTIMVA01GBQ664S, XTIMVA01GBQ667S, XTIMVA01GRA188S, XTIMVA01GRA657S, XTIMVA01GRA659S, XTIMVA01GRA664N, XTIMVA01GRA664S, XTIMVA01GRA667S, XTIMVA01GRM657S, XTIMVA01GRM659

SPDYNCBRTINBGD, SPDYNCBRTINBGR, SPDYNCBRTINBHR, SPDYNCBRTINBHS, SPDYNCBRTINBIH, SPDYNCBRTINBLR, SPDYNCBRTINBLZ, SPDYNCBRTINBMU, SPDYNCBRTINBOL, SPDYNCBRTINBRA, SPDYNCBRTINBRB, SPDYNCBRTINBRN, SPDYNCBRTINBTN, SPDYNCBRTINBWA, SPDYNCBRTINCAF, SPDYNCBRTINCAN, SPDYNCBRTINCEB, SPDYNCBRTINCHE, SPDYNCBRTINCHI, SPDYNCBRTINCHL, SPDYNCBRTINCHN, SPDYNCBRTINCIV, SPDYNCBRTINCMR, SPDYNCBRTINCOD, SPDYNCBRTINCOG, SPDYNCBRTINCOL, SPDYNCBRTINCOM, SPDYNCBRTINCPV, SPDYNCBRTINCRI, SPDYNCBRTINCSS, SPDYNCBRTINCUB, SPDYNCBRTINCUW, SPDYNCBRTINCYM, SPDYNCBRTINCYP, SPDYNCBRTINCZE, SPDYNCBRTINDEU, SPDYNCBRTINDJI, SPDYNCBRTINDMA, SPDYNCBRTINDNK, SPDYNCBRTINDOM, SPDYNCBRTINDZA, SPDYNCBRTINEAP, SPDYNCBRTINEAS, SPDYNCBRTINECA, SPDYNCBRTINECS, SPDYNCBRTINECU, SPDYNCBRTINEGY, SPDYNCBRTINEMU, SPDYNCBRTINERI, SPDYNCBRTINESP, SPDYNCBRTINEST, SPDYNCBRTINETH, SPDYNCBRTINEUU, SPDYNCBRTINFCS, SPDYNCBRTINFIN, SPDYNCBRTINFJI, SPDYNCBRTINFRA, SPDYNCBRTINFRO, SPDYNCBRTINFSM, SPDYNCBRTINGAB, SPDYNCBRTINGBR, SPDYNCBRTINGEO, SPDYNCBR

SPDYNTFRTINEST, SPDYNTFRTINEUU, SPDYNTFRTINFCS, SPDYNTFRTINFIN, SPDYNTFRTINFRA, SPDYNTFRTINFRO, SPDYNTFRTINGBR, SPDYNTFRTINGRC, SPDYNTFRTINHIC, SPDYNTFRTINHPC, SPDYNTFRTINHUN, SPDYNTFRTINIRL, SPDYNTFRTINISL, SPDYNTFRTINISR, SPDYNTFRTINITA, SPDYNTFRTINJPN, SPDYNTFRTINKOR, SPDYNTFRTINLAC, SPDYNTFRTINLCN, SPDYNTFRTINLDC, SPDYNTFRTINLIC, SPDYNTFRTINLMC, SPDYNTFRTINLMY, SPDYNTFRTINLTU, SPDYNTFRTINLUX, SPDYNTFRTINLVA, SPDYNTFRTINMEA, SPDYNTFRTINMEX, SPDYNTFRTINMIC, SPDYNTFRTINMNA, SPDYNTFRTINNLD, SPDYNTFRTINNOC, SPDYNTFRTINNOR, SPDYNTFRTINNZL, SPDYNTFRTINOEC, SPDYNTFRTINOED, SPDYNTFRTINOSS, SPDYNTFRTINPOL, SPDYNTFRTINPRT, SPDYNTFRTINPSS, SPDYNTFRTINRUS, SPDYNTFRTINSAS, SPDYNTFRTINSMR, SPDYNTFRTINSSA, SPDYNTFRTINSSF, SPDYNTFRTINSST, SPDYNTFRTINSVK, SPDYNTFRTINSVN, SPDYNTFRTINSWE, SPDYNTFRTINTUR, SPDYNTFRTINUMC, SPDYNTFRTINUSA, SPDYNTFRTINWLD, SPDYNTFRTINZAF, SPDYNIMRTINARB, SPDYNIMRTINAUS, SPDYNIMRTINAUT, SPDYNIMRTINBEL, SPDYNIMRTINCAN, SPDYNIMRTINCEB, SPDYNIMRTINCHE, SPDYNIMRTINCHL, SPDYNIMR

SMPOPTOTLARG, SMPOPTOTLARM, SMPOPTOTLASM, SMPOPTOTLATG, SMPOPTOTLAUS, SMPOPTOTLAUT, SMPOPTOTLAZE, SMPOPTOTLBDI, SMPOPTOTLBEL, SMPOPTOTLBEN, SMPOPTOTLBFA, SMPOPTOTLBGD, SMPOPTOTLBGR, SMPOPTOTLBHR, SMPOPTOTLBHS, SMPOPTOTLBIH, SMPOPTOTLBLR, SMPOPTOTLBLZ, SMPOPTOTLBMU, SMPOPTOTLBOL, SMPOPTOTLBRA, SMPOPTOTLBRB, SMPOPTOTLBRN, SMPOPTOTLBTN, SMPOPTOTLBWA, SMPOPTOTLCAF, SMPOPTOTLCAN, SMPOPTOTLCEB, SMPOPTOTLCHE, SMPOPTOTLCHI, SMPOPTOTLCHL, SMPOPTOTLCHN, SMPOPTOTLCIV, SMPOPTOTLCMR, SMPOPTOTLCOD, SMPOPTOTLCOG, SMPOPTOTLCOL, SMPOPTOTLCOM, SMPOPTOTLCPV, SMPOPTOTLCRI, SMPOPTOTLCSS, SMPOPTOTLCUB, SMPOPTOTLCUW, SMPOPTOTLCYM, SMPOPTOTLCYP, SMPOPTOTLCZE, SMPOPTOTLDEU, SMPOPTOTLDJI, SMPOPTOTLDMA, SMPOPTOTLDNK, SMPOPTOTLDOM, SMPOPTOTLDZA, SMPOPTOTLEAP, SMPOPTOTLEAS, SMPOPTOTLECA, SMPOPTOTLECS, SMPOPTOTLECU, SMPOPTOTLEGY, SMPOPTOTLEMU, SMPOPTOTLERI, SMPOPTOTLESP, SMPOPTOTLEST, SMPOPTOTLETH, SMPOPTOTLEUU, SMPOPTOTLFCS, SMPOPTOTLFIN, SMPOPTOTLFJI, SMPOPTOTLFRA, SMPOPTOTLFRO, SMPOPTOTLFSM, SMPOPTOTLGAB, SMPOPT

IDS1MNIAOGG3P, IDS1MNIAOGG4T, IDS1MNIAOGG4U, IDS1MNIAOGG4W, IDS1MNIAOGG4Y, IDS1MNIAOGG5R, IDS1MNIAOGGAT, IDS1MNIAOGGBE, IDS1MNIAOGGCZ, IDS1MNIAOGGDE, IDS1MNIAOGGDK, IDS1MNIAOGGES, IDS1MNIAOGGFI, IDS1MNIAOGGFR, IDS1MNIAOGGGB, IDS1MNIAOGGGR, IDS1MNIAOGGHU, IDS1YMAORIGG1N, IDS1YMAORIGG1Z, IDS1YMAORIGG3C, IDS1YMAORIGG3P, IDS1YMAORIGG4T, IDS1YMAORIGG4U, IDS1YMAORIGG4W, IDS1YMAORIGG4Y, IDS1YMAORIGG5R, IDS1YMAORIGGAT, IDS1YMAORIGGBE, IDS1YMAORIGGCZ, IDS1YMAORIGGDE, IDS1YMAORIGGDK, IDS1YMAORIGGES, IDS1YMAORIGGFI, IDS1YMAORIGGFR, IDS1YMAORIGGGB, IDS1YMAORIGGGR, IDS1YMAORIGGHU, IDSGAMRIAO1N, IDSGAMRIAO1Z, IDSGAMRIAO3C, IDSGAMRIAO3P, IDSGAMRIAO4T, IDSGAMRIAO4U, IDSGAMRIAO4W, IDSGAMRIAO4Y, IDSGAMRIAO5R, IDSGAMRIAOAT, IDSGAMRIAOBE, IDSGAMRIAOCZ, IDSGAMRIAODE, IDSGAMRIAODK, IDSGAMRIAOES, IDSGAMRIAOFI, IDSGAMRIAOFR, IDSGAMRIAOGB, IDSGAMRIAOGR, IDSGAMRIAOHU, IDSGAMRINI1N, IDSGAMRINI1Z, IDSGAMRINI3C, IDSGAMRINI3P, IDSGAMRINI4T, IDSGAMRINI4U, IDSGAMRINI4W, IDSGAMRINI4Y, IDSGAMRINI5R, IDSGAMRINIAT, IDSGA

In [143]:
len(series_df)

8444

In [159]:
len(series_df)

156565

In [162]:
series_df[0]

,realtime_start,realtime_end,date,value,series
0,2012-04-26,9999-12-31,1957-07-01,NaN,4BIGEUROREC
1,2012-04-26,9999-12-31,1957-08-01,NaN,4BIGEUROREC
2,2012-04-26,9999-12-31,1957-09-01,NaN,4BIGEUROREC
3,2012-04-26,9999-12-31,1957-10-01,NaN,4BIGEUROREC
4,2012-04-26,9999-12-31,1957-11-01,NaN,4BIGEUROREC
...,...,...,...,...,...
1141,2022-12-08,9999-12-31,2022-04-01,0.0,4BIGEUROREC
1142,2022-12-08,9999-12-31,2022-05-01,0.0,4BIGEUROREC
1143,2022-12-08,9999-12-31,2022-06-01,0.0,4BIGEUROREC
1144,2022-12-08,9999-12-31,2022-07-01,0.0,4BIGEUROREC


In [ ]:
pd.concat(series_df).to_parquet('/Users/jonathanl/Workspace/quant-workspace/experimental-fred/data/series-data/', partition_cols=['series'])

In [161]:
pd.to_pickle(pd.concat(series_df), open('/Users/jonathanl/Workspace/quant-workspace/experimental-fred/data/series-checkpoint-full.pkl', 'wb'))

In [156]:
pd.to_pickle(pd.concat(series_df), open('/Users/jonathanl/Workspace/quant-workspace/experimental-fred/data/series-checkpoint-1.pkl', 'wb'))

In [ ]:
# Storage: 
# (a) Category tree
# (b) Series metadata
# (c) Series data (i.e. parquet load)

In [ ]:
# Retrieval from Storage
# (a) for series: initial numbers
# (b) for series: latest revisions

In [144]:
500000 / 120 / 60

69.44444444444444

In [145]:
69/ 24

2.875